In [3]:
import numpy as np
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload
import nibabel as nib
import pandas as pd
from IPython.display import clear_output
import importlib
from skimage import morphology
from skimage.segmentation import find_boundaries
import porespy as ps
from skimage.morphology import disk, binary_dilation
from mpl_toolkits import mplot3d
import math
import ast
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import json

In [33]:
f = open("/Users/dvarelat/Documents/MASTER/TFM/methods/specimens.json")
data = json.load(f)
data

{'stage1': ['0806_E5', '0504_E1', '0208_E2', '0521_E4', '0521_E2'],
 'stage2': ['0516_E2', '0503_E1', '0209_E1', '0520_E2', '0208_E3'],
 'stage3': ['0209_E2', '0308_E3', '0503_E2', '0516_E3'],
 'stage4': ['0521_E1', '0523_E1', '0806_E3', '0806_E4', '0806_E6'],
 'stage5': ['0401_E3', '0123_E1', '0122_E1', '0518_E3', '0521_E3'],
 'stage6': ['0308_E2',
  '0401_E1',
  '0401_E2',
  '0502_E1',
  '0517_E2',
  '0806_E1',
  '0806_E2'],
 'stage7': ['0308_E4', '0403_E2', '0404_E2', '0516_E5', '0517_E4'],
 'stage8': ['0402_E1', '0402_E2', '0516_E4', '0517_E1', '0518_E2'],
 'stage9': ['0119_E1',
  '0404_E1',
  '0515_E2',
  '0516_E1',
  '0518_E1',
  '0520_E1',
  '0520_E5'],
 'stage10': ['0515_E1', '0520_E4']}

In [39]:
d = {'0806_E5':1, '0504_E1':1, '0208_E2':1, '0521_E4':1, '0521_E2':1, '0516_E2':2, '0503_E1':2, '0209_E1':2, '0520_E2':2, '0208_E3':2, 
 '0209_E2':3, '0308_E3':3, '0503_E2':3, '0516_E3':3, '0521_E1':4, '0523_E1':4, '0806_E3':4, '0806_E4':4, '0806_E6':4, 
 '0401_E3':5,'0401_E1':6,  '0404_E2':7, '0517_E1':8, '0520_E1':9, '0520_E4':10}

In [6]:
features_all = "/Users/dvarelat/Documents/MASTER/TFM/DATA/EXTRACTION/features"
e = folder.split("_")[1] + "_" + folder.split("_")[2]
fr = f"/Users/dvarelat/Documents/MASTER/TFM/DATA/EXTRACTION/{folder}/{e}_cell_properties_radiomics.csv"

In [27]:
FOLDERS = [
    "1_20190504_E1",
    "7_20190404_E2",
    "10_20190520_E4",
    "3_20190516_E3",
    "4_20190806_E3",
    "2_20190520_E2",
    "5_20190401_E3",
    "8_20190517_E1",
    "9_20190520_E1",
    '6_20190401_E1',
]
ESPECIMENS10 = [folder.split("_")[1] + "_" + folder.split("_")[2] for folder in FOLDERS]
list_dfs_radio =[]
for i,folder in enumerate(FOLDERS):
    e = folder.split("_")[1] + "_" + folder.split("_")[2]
    fr = f"/Users/dvarelat/Documents/MASTER/TFM/DATA/EXTRACTION/10MEAN/{folder}/{e}_cell_properties_radiomics.csv"
    dfr= pd.read_csv(fr)
    dfr["especimen"] = e
    list_dfs_radio.append(dfr)

In [28]:
flat_list = [item for sublist in list(data.values()) for item in sublist]
for e in flat_list:
    if f"2019{e}" not in ESPECIMENS10:
        f = os.path.join(features_all, f"2019{e}_cell_properties_radiomics.csv")
        if os.path.isfile(f): 
            print(e)
            dfr= pd.read_csv(f)
            dfr["especimen"] = f"2019{e}"
            list_dfs_radio.append(dfr)
    else:
        print("here")

0806_E5
here
0208_E2
0521_E4
0521_E2
0516_E2
0503_E1
0209_E1
here
0208_E3
0209_E2
0308_E3
0503_E2
here
0521_E1
0523_E1
here
0806_E4
0806_E6
here
here
here
here
here
here


In [45]:
dfall_r = pd.concat(list_dfs_radio)
dfall_r["stageNUMBER"] = dfall_r["especimen"].apply(lambda x: d[x.replace("2019","")])
dict_lines = dict(zip(dfall_r.lines.unique(),  ["other", "paraxial", "somatic", "splanchnic", "myocardium", "myocardium", "splanchnic"]) ) 
dfall_r["name_lines"] = dfall_r["lines"].apply(lambda x: dict_lines[x])

In [43]:
dfall_r.lines.unique()

array([ 0.,  6.,  4.,  5.,  1., 10., 11.])

In [ ]:
dc = {}
for i,folder in enumerate(FOLDERS):
    e = folder.split("_")[1] + "_" + folder.split("_")[2]
    df = dfall_r[dfall_r.especimen == e]
    d[e] = df.groupby(["name_limproved_lines"]).count()["volumes"].to_dict()
    d[e]["stageNUMBER"] = dict_order[i]
    
    d[e]["stage"] = str(dict_order[i])
    d[e]["d1d2"] = float(dfd.d1d2[dfd.Ename == e.replace("2019", "")].values)
    d[e]["totalNum"] = np.sum(list(df.groupby(["name_limproved_lines"]).count()["volumes"].to_dict().values()))
    d[e]["R"] = df[df.RL == 0].shape[0]
    d[e]["L"] = df[df.RL == 1].shape[0]

In [67]:
dc = {}
for df in list_dfs_radio:
    e = df.especimen[0]
    df["name_lines"] = df["lines"].apply(lambda x: dict_lines[x])
    dc[e] = df.groupby(["name_lines"]).count()["volumes"].to_dict()
    dc[e]["stageNUMBER"] = d[e.replace("2019","")]
    dc[e]["totalNum"] = np.sum(list(df.groupby(["name_lines"]).count()["volumes"].to_dict().values()))

In [82]:
df_numbers = pd.DataFrame(dc).T

df_numbers["myo/total"] = df_numbers.apply(lambda x: x["myocardium"]/x["totalNum"], axis=1)

dforder_numbers = df_numbers.sort_values(by=["stageNUMBER"])
dforder_numbers

,myocardium,other,paraxial,somatic,splanchnic,stageNUMBER,totalNum,myo/total
20190504_E1,581,847,84,330,795,1,2637,0.220326
20190521_E2,878,2430,124,411,936,1,4779,0.183720
20190208_E2,216,2502,58,153,253,1,3182,0.067882
20190806_E5,530,1696,122,225,494,1,3067,0.172807
20190521_E4,1018,1346,118,302,826,1,3610,0.281994
20190520_E2,770,1535,29,240,725,2,3299,0.233404
20190208_E3,1124,2824,157,618,860,2,5583,0.201325
20190209_E1,813,2408,92,375,788,2,4476,0.181635
20190503_E1,975,3469,124,502,832,2,5902,0.165198
20190516_E2,1055,3192,164,476,1246,2,6133,0.172020


In [85]:
melted = pd.melt(dforder_numbers, id_vars=["stageNUMBER"],  #stage
        value_vars=["myocardium","myo/total"])
melted = melted[melted.stageNUMBER != 10]
melted.head()

,stageNUMBER,variable,value
0,1,myocardium,581.0
1,1,myocardium,878.0
2,1,myocardium,216.0
3,1,myocardium,530.0
4,1,myocardium,1018.0


In [86]:
fig = px.line(melted[melted.variable == "myo/total"], x="stageNUMBER", y="value",
              markers=True ,template="plotly_white")
fig

In [87]:
fig = px.line(melted[melted.variable == "myocardium"], x="stageNUMBER", y="value",
              markers=True ,template="plotly_white")
fig